Soft Computing, Simona Prokić, SW16/2015
Sudoku Solver

In [ ]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf

# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras import backend as K

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def gray_image(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def treshold(image):
#     return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 3)
#     return cv2.threshold(image,127,255,cv2.THRESH_BINARY)
#     return cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    return (image > 250) * 255

def display_image(image, color=False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')
        
def blur_image(image, k_size=3):
    return cv2.blur(image, (k_size,k_size))

def dilate_image(image):
    kernel = np.ones((3, 3))
    return cv2.dilate(image, kernel, iterations=2)

def erode_image(image):
    kernel = np.ones((3, 3))
    return cv2.erode(image, kernel, iterations=1)

def invert_image(image):
    return 255-image

def process_hough_result(result):
    lines = []
    for line in hough_result:
        lines.append(line[0])
        lines = sorted(lines, key=lambda line:line[0])
    return lines

def print_grid(arr): 
    for i in range(9): 
        if i%3==0:
            print("___________________")
        for j in range(9):
            if j==0:
                print("|",end='')
            if j%3==2:
                print(arr[i][j], end="|")
            else:
                print(arr[i][j], end=" ")
        print()
    print("___________________")


In [ ]:
def canny(image, ratio=3, kernel_size=5):
    return cv2.Canny(np.uint8(image), 255/ratio, 255, kernel_size)

In [ ]:
path = 'images/sudoku4.png'
img = load_image(path)
image = gray_image(img)
image = blur_image(image)
image = canny(image)
image = dilate_image(image)

display_image(image)



In [ ]:
hough_result = cv2.HoughLines(image, 1, np.pi/180, 600)
lines = process_hough_result(hough_result)
final_lines = []

pos_hor = 0
pos_ver = 0
first_hor_line = True
first_ver_line = True

for line in lines:
    rho, theta = line
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    
    if (not first_hor_line) and (not first_ver_line):
        if b==1:
            if rho-pos_hor>10:
                pos_hor = rho
                cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
                final_lines.append([rho, theta, 0])
        elif b==0:
            if rho-pos_ver>10:
                pos_ver = rho
                cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
                final_lines.append([rho, theta, 1])
    elif b==1:
        if first_hor_line:
            final_lines.append([rho, theta, 0])
            pos_hor = rho
            first_hor_line = False
            cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
            continue
    elif b==0:
        if first_ver_line:
            final_lines.append([rho, theta, 1])
            pos_ver = rho
            first_ver_line = False
            cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
            continue
    

cv2.imwrite('houghlines.jpg',img)
image_lines = load_image('houghlines.jpg')
display_image(image_lines)

In [ ]:
points = []
for hor_line in final_lines:
    if hor_line[2] == 0:
        for ver_line in final_lines:
            if ver_line[2] == 1:
                rho1, theta1 = hor_line[0], hor_line[1]
                rho2, theta2 = ver_line[0], ver_line[1]
                xy = np.array([[np.cos(theta1), np.sin(theta1)], [np.cos(theta2), np.sin(theta2)]])
                rho = np.array([rho1,rho2])
                result = np.linalg.solve(xy, rho)
                points.append(result)

rectangles = []
if len(points) == 100:
    img = load_image(path)
    for i in range(0,9):
        for j in range(0,9):
            y1=int(points[j+i*10][1]+5)
            y2=int(points[j+i*10+11][1]-5)
            x1=int(points[j+i*10][0]+5)
            x2=int(points[j+i*10+11][0]-5)
            cv2.rectangle(img,(x1,y1),(x2, y2),(255,0,0),2)
            rectangles.append([x1,y1,x2,y2])
            cv2.imwrite('rectangles.jpg',img)
            image_rectangles = load_image('rectangles.jpg')
            display_image(image_rectangles)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
batch_size = 128
num_classes = 10
epochs = 20
# input image dimensions
img_rows, img_cols = 28, 28

# Reshaping the array to 4-dims so that it can work with the Keras API
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# We may always experiment with kernel size, pool size, activation functions, dropout rate
# and number of neurons in the first Dense layer to get a better result.
model = Sequential() # Creating a Sequential Model and adding the layers
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# We can experiment with the optimizer, loss function, metrics, and epochs.
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
model.save('model.h5')

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from keras.models import load_model
model = load_model('model.h5')

In [ ]:
sudoku_img = load_image(path)
predictions = []

for rect in rectangles:
    crop_img = sudoku_img[rect[0]:rect[2], rect[1]:rect[3]]

    h, w, c = crop_img.shape
    im = np.zeros(shape=(h+10, w+10, c), dtype='uint8') + 255
    im[:h, :w, :] = crop_img
    im[:5, :, :] = 255
    im[:, :5, :] = 255

    im = gray_image(cv2.resize(im, (28,28)))
    im = invert_image(im)
    
    if cv2.countNonZero(im) == 0:
        predictions.append(0)
    else:
        pred = model.predict(im.reshape(1,28,28,1))
        predictions.append(pred.argmax())


In [ ]:
grid=[[0 for x in range(9)]for y in range(9)]
for i in range(9):
    for j in range(9):
        grid[i][j] = predictions[i+j*9]
print_grid(grid)

In [ ]:
# BACKTRACKING ALGORITHM     
def find_empty_location(arr,l): 
    for row in range(9): 
        for col in range(9): 
            if(arr[row][col]==0): 
                l[0]=row 
                l[1]=col 
                return True
    return False

def used_in_row(arr,row,num): 
    for i in range(9): 
        if(arr[row][i] == num): 
            return True
    return False

def used_in_col(arr,col,num): 
    for i in range(9): 
        if(arr[i][col] == num): 
            return True
    return False

def used_in_box(arr,row,col,num): 
    for i in range(3): 
        for j in range(3): 
            if(arr[i+row][j+col] == num): 
                return True
    return False

def check_location_is_safe(arr,row,col,num): 
    safe = not used_in_row(arr,row,num) and not used_in_col(arr,col,num) and not used_in_box(arr,row - row%3,col - col%3,num) 
    return safe

def solve_sudoku(arr): 
    l=[0,0] # list that keeps the record of row and col in find_empty_location()     
         
    if(not find_empty_location(arr,l)): 
        return True
      
    # row and col of empty cell
    row=l[0] 
    col=l[1] 
      
    for num in range(1,10):
        if(check_location_is_safe(arr,row,col,num)):     
            arr[row][col] = num
            if(solve_sudoku(arr)):
                return True
            arr[row][col] = 0 # failure->empty cell and try again 
 
    return False # this triggers backtracking

# assigning values to the grid 
# grid=[[0,0,6,3,1,0,0,0,0], 
#       [0,2,3,0,8,7,0,0,0], 
#       [0,0,9,0,6,0,0,2,8], 
#       [2,9,0,0,0,0,0,6,0], 
#       [0,0,0,0,0,0,0,0,0], 
#       [0,5,0,0,0,0,0,4,2], 
#       [7,6,0,0,3,0,2,0,0], 
#       [0,0,0,6,9,0,1,5,0], 
#       [0,0,0,0,2,5,4,0,0]]

if solve_sudoku(grid):
    print_grid(grid)
else:
    print('no solution')

